# Volcanoes and Emissions

### Building a visualization similar to this app http://volcano.si.axismaps.io/ 

### Imports and function definitions

In [309]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template for Equirectangular Panoramas

In [310]:
EquiRectPanoramaTemplate = """mesh
{  
    data  panaorama ./modules/$folderName/sphere.obj
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec4f ColorMultiplier     $ColorMultiplier
            vec3f RotationAxis        $RotationAxis
            vec1f RotationAngle       $RotationAngle
        }
    }
    pass
    {
        useDataObject panaorama
        shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/equirectangularPanorama.vs
                fragmentShader ./modules/$folderName/equirectangularPanorama.fs
                
                texture panorama ./modules/$folderName/$panorama
                {
                }
                

                parameter1f Scale $Scale
                
                stateManagerVar __objectName__.ColorMultiplier      ColorMultiplier
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.Alpha                uv_alpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
}"""

## Conf for Volcanoes and Emission data

In [311]:
VETemplate='''mesh
{
    data SunData    ./modules/$folderName/MESA_1M_pre_ms_to_wd.raw
    cullRadius $cr
    glslVersion 330


    propertyCollection
    {
        __objectName__
        {
            vec1f simBindRealtime 0. #bind the time of this model to the actual time in Uniview (0 == no, 1 == yes)
            vec1f simUseTime 4500. #if simBindReadtime == 0, then this is the time to use (Myr)
            vec1f simRealtimestart 0. #Myr
            vec1f simRealtimeend 12000. #Myr
            vec1f simShowstart 0.0
            vec1f simDtmin 0.01


        }

    }


#####################################################
##Sun

    pass #Corona
    {
        passEnable true

        useDataObject SunData

        shader
        {
            type defaultMeshShader
            {


                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simRealtimestart simRealtimestart  
                StateManagerVar __objectName__.simRealtimeend simRealtimeend 
                StateManagerVar __objectName__.simDtmin simdtmin


                vertexShader ./modules/$folderName/VE_shader.vs
                fragmentShader ./modules/$folderName/VE_shader.fs                
                geometryShader ./modules/$folderName/VE_shader.gs



                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                }
             }
        }
    }


}

'''

### Equirectangular Panorama Class

In [312]:
class EquiRectPanorama():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("equirectangularPanorama.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("equirectangularPanorama.fs",dataFolder)
        self.panorama = "UNINITIALIZED"
        self.ColorMultiplier = "1 1 1 1"
        self.Scale = 1
        self.RotationAxis = "1 0 0"
        self.cr = 1000
        self.RotationAngle = 0
    def setPanorama(self, panorama):
        uvmod.Utility.ensurerelativepathexsists(panorama,dataFolder)
        self.panorama = panorama
    def setColorMultiplier(self,ColorMultiplier):
        self.ColorMultiplier = " ".join(str(c) for c in ColorMultiplier)
    def setScale(self,Scale):
        self.Scale = Scale
    def setRotationAxis(self,RotationAxis):
        self.RotationAxis = " ".join(str(c) for c in RotationAxis)
    def setRotationAngle(self,RotationAngle):
        self.RotationAngle = RotationAngle
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(EquiRectPanoramaTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 panorama = self.panorama,
                                 ColorMultiplier = self.ColorMultiplier,
                                 Scale = self.Scale,
                                 RotationAxis = self.RotationAxis,
                                 RotationAngle = self.RotationAngle))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

## Volcanoes and Emissions class

In [313]:
class VEclass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("VE_shader.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("VE_shader.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("VE_shader.fs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry("VE.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = "VE.usesconf"
        s = string.Template(VETemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [314]:
EarthMap = EquiRectPanorama(uvmod.OrbitalObject())
VEmodel = VEClass(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [319]:
EarthMap.object.setcameraradius(10)
EarthMap.object.setcoord("SolarSystem")
EarthMap.object.setname("EarthMap")
EarthMap.object.setguiname("/KavliLecture/Schrag/Earth Map")
EarthMap.object.settargetradius(20)
EarthMap.object.showatstartup(False)
EarthMap.setPanorama("BlankMap-World6-Equirectangular.png")
EarthMap.setColorMultiplier((1.,1.,1.,1.))
EarthMap.setScale(100)
EarthMap.cr = 10000

VEmodel.object.setcameraradius(50)
VEmodel.object.setcoord("SolarSystem")
VEmodel.object.setname("VolcanoesEmissions")
VEmodel.object.setguiname("/KavliLecture/Schrag/Volcanoes and Emissions")
#star.object.settargetradius(500)
#star.object.setscalefactor(10.0)
VEmodel.object.showatstartup(False)


modinfo.setname("Volcanoes and Emissions")
#modinfo.setauthor("Mark SubbaRao<sup>1</sup> and Lisa Kaltenegger<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Cornell University<br/>panorama from:The Reefs of Belize: <br/> A co-production by the Massachusetts Institute of Technology (MIT) and the Charles Hayden Planetarium, Museum of Science")
modinfo.cleardependencies()
#modinfo.adddependency("AlphaCent")
#modinfo.setdesc("A spherical panorama of a coral reef")
#modinfo.setthumbnail("data/TheReefsOfBelize_VR_002064.png")
modinfo.setversion("1.0")

generator.generate("VolcanoesEmissions",[],[EarthMap, VEmodel],modinfo)
uvmod.Utility.senduvcommand(EarthMap.object.name+".reload")
uvmod.Utility.senduvcommand(VEmodel.object.name+".reload")

Unable to connect to Uniview
Unable to connect to Uniview


### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [316]:
uvmod.Utility.senduvcommand(EarthMap.object.name+".reload")
uvmod.Utility.senduvcommand(VEmodel.object.name+".reload")

Unable to connect to Uniview
Unable to connect to Uniview


Copy modified Shader files and reload

In [317]:
from config import Settings


In [318]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+VEmodel.object.name)
uvmod.Utility.senduvcommand(VEmodel.object.name+".reload")

Unable to connect to Uniview
